In [114]:
import pandas as pd
import numpy as np
from os import listdir
from os.path import isfile, join
import re
from PIL import Image
from sklearn.feature_extraction import image as sklim
from skimage.util import shape
import matplotlib.pyplot as plt


In [115]:
data = 'data-99spots/'
files = [f for f in listdir(data) if isfile(join(data, f)) and f.endswith('.csv')]

# for file in files:
#     with open(data+files[1], 'rb') as csvfile:
#         reader = csv.reader(csvfile, delimiter=',', quotechar='|')
#         for row in reader:
#             print ', '.join(row)

csv_file = data + files[1]
df = pd.read_csv(csv_file)
# saved_column = df.column_name #you can also use df['column_name']
# print df['Cell_ID']
# a =df[:3]
xy = np.concatenate((np.reshape(df.Cell_X, (-1,1)), np.reshape(df.Cell_Y, (-1,1))),axis=1)
features = {}
# biomarker = []

def determineCohort(spot_name):
    cohort_1 = ['000', '005', '026', '031', '046', '055', '060', '081', '086']
    cohort_2 = ['001', '006', '011', '016', '025', '030', '043', '045', '056',
         '061', '066', '076', '080', '085', '096']
    cohort_3 = ['002', '007', '012', '017', '020', '024', '029', '032', '034',
        '036', '039', '044', '049', '052', '057', '062', '067', '071', '072',
        '079', '084', '089', '091', '095']
    cohort_4 = ['003', '008', '013', '015', '018', '021', '023', '028', '033',
        '038', '041', '048', '053', '058', '063', '065', '068', '070', '073',
        '078', '083', '088', '090', '093']
    if spot_name in cohort_1:
        return 1
    elif spot_name in cohort_2:
        return 2
    elif spot_name in cohort_3:
        return 3
    elif spot_name in cohort_4:
        return 4
    else:
        print 'Spot does not belong to any cohorts!'
        
for idx, col in enumerate(list(df)):
    
    pass
    
# #      m = re.search('^[^_]+(?=_)', col)
#     biomarker.append(tuple(col.split('_')[0:2]))
#     features[biomarker[idx]] = df[col].values
        
     
     # print m.group(0)
biomarker, location, col = zip(*[(col.split('_')[0], col.split('_')[1], col) for col in list(df) if col.split('_')[0] != 'Cell'])
# b, l = [col.spl]
# biomarker = [m.group(0) for col in list(df) for m in [re.search('^[^_]+(?=_)', col)] if col != 'Cell']
uniq_biomarker = list(set(biomarker))
uniq_location = list(set(location))

k = 0
for i, bio in enumerate(uniq_biomarker):
    features[bio] ={}
    for j, loc in enumerate(uniq_location):
#         features[bio][loc] = df[col[i*len(uniq_location) + j]].values
        features[bio][loc] = df[col[k]].values
#         print i*(len(uniq_location)) + j
        k+=1
        
#         print i*len(uniq_location) + j
        
# for i in range(len(col)):
#     features[biomarker[i]][location[i]] = df[col[i]]
    
        


# biomarker[:] = list(set((bio for bio in biomarker if bio != 'Cell')))

# for idx, col in enumerate(list(df)):
#     print idx, col
    

# filename = '011_Quant.csv'
# spot_name = re.search('\d{2,3}', filename).group(0)





In [116]:
print biomarker[4]
print location[2]
print col[2]
print uniq_biomarker
# print uniq_location

Her2
Mem
EGFR_Mem_Quant
['PR', 'Her2', 'PCAD', 'EGFR', 'ER']


In [117]:
for i in range(5):
    print features.keys()[i]
    print features[features.keys()[i]].keys()

    

PR
['Mem', 'Nuc', 'Cyt']
Her2
['Mem', 'Nuc', 'Cyt']
PCAD
['Mem', 'Nuc', 'Cyt']
EGFR
['Mem', 'Nuc', 'Cyt']
ER
['Mem', 'Nuc', 'Cyt']


In [118]:
print biomarker[2]
spot_num = '011'

EGFR


In [119]:
image_dir = 'images'
a=[join(image_dir,f) for f in listdir(image_dir) if uniq_biomarker[0] in f]

In [120]:
image = [join(a[0],f) for f in listdir(a[0]) if spot_num in f]
print image

['images/PR-allTissue/PR_AFRemoved_011.tif']


In [121]:
im = Image.open(image[0]).convert("L")

In [112]:
def im2double(im):
    min_val = np.min(im.ravel())
    max_val = np.max(im.ravel())
    out = (im.astype('float') - min_val) / (max_val - min_val)
    return out
i = np.array(im)

i = im2double(i)

In [12]:
# import numpy
# import PIL
 
# # Convert Image to array
# img = PIL.Image.open("foo.jpg").convert("L")
# arr = numpy.array(img)
 
# # Convert array to Image
# img = PIL.Image.fromarray(arr)

In [13]:
tilesize = 256

# Generates completely overlapping patches
patches = sklim.extract_patches_2d(i, (256, 256), max_patches=100, random_state=None )

In [14]:
# patchifiy image!!
%timeit for x in range(100): shape.view_as_blocks(i,(256, 256))
%timeit for x in range(100): shape.view_as_windows(i,(256, 256), 256)
patches = shape.view_as_windows(i,(256, 256), 64)

100 loops, best of 3: 2.05 ms per loop
100 loops, best of 3: 2.92 ms per loop


In [19]:
# show_subset_patches
print patches.shape
k = 1
for i in range(5):
    for j in range(5):
        plt.subplot(5, 5, k)
        a = patches[i,j,:,:]
        plt.imshow(a)
        k+=1
plt.show()

(29, 29, 256, 256)


In [31]:
def patchify(img, patch_shape):
    img = np.ascontiguousarray(img)  # won't make a copy if not needed
    X, Y = img.shape
    x, y = patch_shape
    shape = ((X-x+1), (Y-y+1), x, y) # number of patches, patch_shape
    # The right strides can be thought by:
    # 1) Thinking of `img` as a chunk of memory in C order
    # 2) Asking how many items through that chunk of memory are needed when indices
    #    i,j,k,l are incremented by one
    strides = img.itemsize*np.array([Y, 1, Y, 1])
    return np.lib.stride_tricks.as_strided(img, shape=shape, strides=strides)
# %load 'src/python/classes/TissueMicroArray.py'

In [94]:
im = np.arange(16).reshape(4,4)
im3 = np.arange(16*3).reshape((4,4,3))
print im

[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]
 [12 13 14 15]]


In [103]:
p = shape.view_as_windows(im,(1,1))
p3 = shape.view_as_windows(im3, (2,2,3),(2,2,1))


In [104]:
p.shape
p3.shape

(2, 2, 1, 2, 2, 3)

In [53]:
p3[0,0,0,:,:,1]

array([[ 1,  4,  7, 10],
       [25, 28, 31, 34],
       [49, 52, 55, 58],
       [73, 76, 79, 82]])

In [110]:
if not None:
    print 'hi'

hi


In [22]:
patch_shape = (256, 256, 3)

overlap = 1

step = tuple((patch_shape[0]-overlap, patch_shape[1]-overlap, 1))
print step


(255, 255, 1)
